In [1]:
import numpy as np

In [2]:
s1 = "amer"
s2 = "albert"

In [18]:
def levenshtein_distance(s1, s2):
    """
    Compute the levenshtein distance between two string given in parameters
    Operations: insert (cost 1), delete (cost 1), replace (cost 1)
    
    Inputs:
    -------
    s1, s2:      strings
                 two strings to compare
    Output:
    -------
    mat[-1, -1]: float64
                 levenshtein distance between s1 and s2
    """
    l1 = len(s1)
    l2 = len(s2)
    mat = np.zeros((l1, l2))
    mat[:, 0] = np.arange(l1)
    mat[0, :] = np.arange(l2)
    for i in range(1, l1):
        for j in range(1, l2):
            if s1[i] == s2[j]:
                mat[i,j] = min(mat[i-1, j] + 1, mat[i, j-1] + 1, mat[i-1, j-1])
            else:
                mat[i,j] = min(mat[i-1, j] + 1, mat[i, j-1] + 1, mat[i-1, j-1] +1)
    return mat[-1, -1]

In [19]:
res = levenshtein_distance(s1, s2)

In [20]:
res.dtype

dtype('float64')

In [21]:
levenshtein_distance('adsasfasf', 'dad oas eh')

7.0

# Levenshtein automaton

In [25]:
class LevenshteinAutomaton:
    def __init__(self, string, n):
        self.string = string
        self.max_edits = n

    def start(self):
        return range(len(self.string)+1)

    def step(self, state, c):
        new_state = [state[0]+1]
        for i in range(len(state)-1):
            cost = 0 if self.string[i] == c else 1
            new_state.append(min(new_state[i]+1, state[i]+cost, state[i+1]+1))
        return [min(x,self.max_edits+1) for x in new_state]

    def is_match(self, state):
        return state[-1] <= self.max_edits

    def can_match(self, state):
        return min(state) <= self.max_edits

    def transitions(self, state):
        return set(c for (i,c) in enumerate(self.string) if state[i] <= self.max_edits)

In [41]:
counter = [0] # list is a hack for mutable lexical scoping
states = {}
transitions = []
matching = []

lev = LevenshteinAutomaton("woof", 2)

def explore(state):
    key = tuple(state) # lists can't be hashed in Python so convert to a tuple
    if key in states: return states[key]
    i = counter[0]
    counter[0] += 1
    states[key] = i
    if lev.is_match(state): matching.append(i)
    for c in lev.transitions(state) | set(['*']):
        newstate = lev.step(state, c)
        j = explore(newstate)
        transitions.append((i, j, c))
    return i

explore(lev.start())

0

In [37]:
counter = [0] # list is a hack for mutable lexical scoping
states = {}
transitions = []
matching = []

lev = LevenshteinAutomaton("woof", 1)


In [39]:
start = lev.start()

In [40]:
tuple(start)

(0, 1, 2, 3, 4)

In [28]:
states

{(0, 1, 2, 3, 4): 0,
 (1, 0, 1, 2, 2): 7,
 (1, 1, 1, 2, 2): 15,
 (1, 1, 2, 2, 2): 1,
 (2, 1, 0, 1, 2): 10,
 (2, 1, 1, 2, 2): 8,
 (2, 1, 2, 2, 2): 3,
 (2, 2, 1, 0, 1): 11,
 (2, 2, 1, 1, 1): 14,
 (2, 2, 1, 1, 2): 9,
 (2, 2, 1, 2, 2): 4,
 (2, 2, 2, 1, 0): 13,
 (2, 2, 2, 1, 1): 12,
 (2, 2, 2, 1, 2): 5,
 (2, 2, 2, 2, 1): 6,
 (2, 2, 2, 2, 2): 2}

In [29]:
transitions

[(2, 2, '*'),
 (1, 2, '*'),
 (3, 2, '*'),
 (4, 2, '*'),
 (5, 2, '*'),
 (6, 2, '*'),
 (5, 6, 'f'),
 (4, 5, 'o'),
 (3, 4, 'o'),
 (1, 3, 'w'),
 (1, 4, 'o'),
 (0, 1, '*'),
 (8, 2, '*'),
 (9, 2, '*'),
 (9, 5, 'o'),
 (9, 6, 'f'),
 (8, 9, 'o'),
 (7, 8, '*'),
 (7, 8, 'w'),
 (10, 9, '*'),
 (12, 2, '*'),
 (12, 6, 'f'),
 (11, 12, '*'),
 (11, 12, 'o'),
 (13, 6, '*'),
 (13, 6, 'f'),
 (11, 13, 'f'),
 (10, 11, 'o'),
 (14, 2, '*'),
 (14, 5, 'o'),
 (14, 6, 'f'),
 (10, 14, 'f'),
 (7, 10, 'o'),
 (0, 7, 'w'),
 (15, 2, '*'),
 (15, 3, 'w'),
 (15, 9, 'o'),
 (0, 15, 'o')]

In [30]:
matching

[6, 11, 12, 13, 14]

In [31]:
counter

[16]